[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ethanjperez/rda/blob/master/rda.ipynb)

# **Tutorial**: How to run Rissanen Data Analysis on your own dataset

This short notebook shows how you can run Rissanen Data Analysis (RDA) on any dataset, using any model of your choice. To perform RDA, you'll need to compute the Minimum Description Length (MDL) of the labels of your dataset given the inputs. Then, you can modify the dataset inputs (i.e., adding or removing certain features, like nouns) and see how the MDL changes. In this notebook, we'll illustrate how to compute MDL of a dataset in the GLUE benchmark, MRPC, by training BERT models, using the HuggingFace Transformers library.

Throughout this tutotial, we'll show you where you need to change the code in order to evaluate the MDL of a different dataset and/or using a different model. You'll only need to change a few lines of code, so it will be quite straightforward. Let's get started!

## Import Dependencies
First, let's import a few basic packages:

In [1]:
import json
import math
import os
import random
import sys

Next, we'll load some library to train models (with PyTorch, Tensorflow, Jax, or anything else). We'll call some model training function from the library (treating it like a black box), in order to train and test on different subsets of the original dataset (which we'll write to file first). We just need the model training function to return us the loss on unseen test examples. The loss value should be the mean squared error for regression tasks and negative log-likelihood otherwise -- we'll use these values to compute the label description lengths.

Here, we train BERT models using HuggingFace Transformers. We've cloned the original repo and modified ~3 lines of code to have the model training function return the test loss after training. You can load this model training function like so:

In [2]:
!pip install datasets
!pip install git+https://github.com/ethanjperez/transformers_rda.git
import transformers.run_glue as train_model

     |████████████████████████████████| 194kB 17.4MB/s 
     |████████████████████████████████| 245kB 43.6MB/s 
     |████████████████████████████████| 112kB 41.3MB/s 
  Cloning https://github.com/ethanjperez/transformers_rda.git to /tmp/pip-req-build-kx6usb6f
  Running command git clone -q https://github.com/ethanjperez/transformers_rda.git /tmp/pip-req-build-kx6usb6f
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 3.2MB 20.2MB/s 
     |████████████████████████████████| 890kB 52.9MB/s 
  Created wheel for transformers: filename=transformers-4.4.0.dev0-cp37-none-any.whl size=1897591 sha256=bd3ba73681857ce9102ff3f45483bbc9093d7f3c54567c684db69279efb2ca50
  Stored in directory: /tmp/pip-ephem-wheel-cache-ab_znymc/wheels/5d/fa/c7/ec6625926de36a024c2510fc7c4e67aed1e6493eb20c82404f
Successfully built transformers
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp3

## Specify RDA Experimental Setup
Next, let's set a few basic variables (with assertion checks) to set how we'll compute RDA:

In [3]:
# The number of blocks N we use when sending labels
# We train N-1 models, since the first block is sent with a uniform prior
num_blocks = 9
assert num_blocks >= 1, 'num_blocks must be >= 1'

# The minimum/maximum number of examples to train models with
min_num_train_samples = 64
max_num_train_samples = float('inf') # use all examples
assert min_num_train_samples >= 1, 'min_num_train_samples must be >= 1'
assert max_num_train_samples >= min_num_train_samples, 'max_num_train_samples must be >= --min_num_train_samples'

# The fraction of examples to split off for validation (the rest are used for training)
val_frac = 0.1
assert 0 <= val_frac < 1, 'val_frac must be >= 0 and < 1'

Now, let's set a few task-specific variables. We need to know the range of possible output values, in order to compute the codelength for sending the first block of labels. For MRPC, there are just two possible labels:

In [4]:
label_range = 2
assert label_range > 0, 'label_range must be > 0'

uniform_prior_nll = -math.log(1. / float(label_range))

For text generation or span prediction, `label_range` will be quite large, as there are many possible outputs. For regression, you'll want to set `label_range` to the size of the interval over which outputs can range, e.g., 3.5 if the range is [1., 4.5]. Speaking of regression, let's set a variable to keep track of whether or not we expect to receive mean squared error values from our model training function (so we'll know to convert MSE values to negative log-likelihoods).

In [5]:
mse = False

Lastly, set the command line arguments you want to use to train your model. We'll need to point the model to the training/validation/test data for each block, but that will change for each block. For now, we can just set those paths to special string `TRAIN_FILE`, `VALIDATION_FILE`, and `TEST_FILE` (we'll replace these with actual file paths later). Here's what this looks like for training a HuggingFace BERT model:

In [6]:
training_args = "--model_name_or_path bert-base-cased --do_train --do_eval --max_seq_length 128 " + \
    "--per_device_train_batch_size 32 --learning_rate 2e-5 --num_train_epochs 3 --output_dir checkpoint " + \
    "--train_file TRAIN_FILE --validation_file VALIDATION_FILE --test_file TEST_FILE --overwrite_output_dir"

Note that `--overwrite_output_dir` will clear the saved results of any previous model training run that saves to the same `--output_dir` (here, `checkpoint/`). This is the behavior that we want, since when we send a new block, we don't want to reload the model, results, etc. for sending an earlier block of data. If you call a different model training function, you'll similarly want to ensure that you aren't accidentally loading the results of previous training runs when you make a new call to the model training function.

## Dataset Setup

Now that we've specified our RDA setup, let's load our dataset into a list of examples (these can have any data type/structure). We load the MRPC data (just the training set for MDL evaluation) like so:

In [7]:
from datasets import load_dataset
dataset = list(load_dataset("glue", "mrpc", split='train'))

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/mrpc/1.0.0/7c99657241149a24692c402a5c3f34d4c9f1df5ac2e4c3759fadea38f6cb29c4. Subsequent calls will reuse this data.


At this point, you can augment or ablate the input data loaded above if desired. We'll just show how to compute MDL on the original MRPC dataset, so we don't do any input modification here.

Next, let's randomly order the dataset examples using a fixed random seed:

In [8]:
seed = 0
rng = random.Random(seed)
rng.shuffle(dataset)

To train the model on different subsets of the data, we'll need a function that saves a list of examples to file, in a format that the model training function can read from. Later, we can point the model training function to different training/validation/test data files as needed, for different blocks. Below, we save instances in a way that is compatible with loading data via HuggingFace datasets:

In [9]:
def save_data(examples, save_file):
    with open(save_file, 'w') as f:
        f.writelines('\n'.join([json.dumps(ex) for ex in examples]))

Let's also set the file extension that we want to use to save data files:

In [10]:
data_file_ext = 'json'

Now, let's compute the starting indices of each block, $t_0, \dots, t_N$:

In [11]:
block_size_logscale_increment = (math.log(min(len(dataset), max_num_train_samples)) - math.log(min_num_train_samples)) / (num_blocks - 1)
block_start_idxs = [0] + [int(round(math.exp(math.log(min_num_train_samples) + (block * block_size_logscale_increment)))) for block in range(num_blocks)]
print('t_0, ..., t_N:', block_start_idxs)

t_0, ..., t_N: [0, 64, 106, 176, 292, 485, 804, 1333, 2211, 3668]


## Computing Negative Log-Likelihoods

Now, we're ready to compute the average negative log-likelihoods (NLLs) for each block.
Let's collect the average NLL for each block in a list, adding the NLL for the first block that we computed earlier:

In [12]:
nlls = [uniform_prior_nll]

Now, we create train/val/test splits for sending each data block after the first, and then send each block one by one by calling `train_model.main()` to train a model on a chunk of the data and get the test loss on a new block.

In [13]:
# Send each block after the first, one by one
for send_block in range(1, num_blocks):
    # Create the train/validation/test data for sending each block
    train_val_dataset = dataset[:block_start_idxs[send_block]] # train/val examples from blocks before the current one
    rng.shuffle(train_val_dataset) # shuffle examples for random train vs. val split
    val_size = int(round(val_frac * len(train_val_dataset))) # compute size of validation set
    block_datasets = { # get list of examples for each split
        'train': train_val_dataset[val_size:],
        'validation': train_val_dataset[:val_size],
        'test': dataset[block_start_idxs[send_block]: block_start_idxs[send_block + 1]],
    }

    # Save train/validation/test data and add data paths to model training arguments
    block_data_dir = 'data/send_block_' + str(send_block) # where we'll save the train/val/test data for sending the current block
    os.makedirs(block_data_dir, exist_ok=True)
    block_training_args = training_args
    for split, block_dataset in block_datasets.items():
        block_split_filepath = os.path.join(block_data_dir, split + '.' + data_file_ext)
        print('Saving data to:', block_split_filepath)
        # Save the data for a single split of data (train, val, or test)
        save_data(block_dataset, block_split_filepath)
        assert (split.upper() + '_FILE') in training_args, 'Expected ' + split.upper() + '_FILE in training_args'
        # Point training arguments to this block's train/val/test data
        block_training_args = block_training_args.replace(split.upper() + '_FILE', block_split_filepath)

    # Set command line args for model training
    sys.argv = [train_model.__file__] + block_training_args.split()
    # Call main function to train model with above args, to get test NLL on this block
    block_nll = train_model.main()
    nlls.append(block_nll)

Saving data to: data/send_block_1/train.json
Saving data to: data/send_block_1/validation.json
Saving data to: data/send_block_1/test.json


Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
[INFO|run_glue.py:198] 2021-03-06 17:57:25,107 >> Training/evaluation parameters TrainingArguments(output_dir=checkpoint, overwrite_output_dir=True, do_train=True, do_eval=True, do_predict=False, evaluation_strategy=EvaluationStrategy.NO, prediction_loss_only=False, per_device_train_batch_size=32, per_device_eval_batch_size=8, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=2e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=3.0, max_steps=-1, lr_scheduler_type=SchedulerType.LINEAR, warmup_ratio=0.0, warmup_steps=0, logging_dir=runs/Mar06_17-57-25_a65b076a9074, logging_strategy=LoggingStrategy.STEPS, logging_first_step=False, logging_steps=500, save_steps=500, save_total_limit=None, no_cuda=False, seed=42, fp16=False, fp16_opt_level=O1, fp16_backend=auto, fp16_full_eval=False, local_rank=-1, tpu_num_core

03/06/2021 17:57:25 - WARNING - datasets.builder -   Using custom data configuration default-63e0d357392494ff


Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-63e0d357392494ff/0.0.0/dc7ee63ec8b554c48ecc5a8a6fbe27af8071408c244e4347cf9222d6206d83a2. Subsequent calls will reuse this data.


[INFO|file_utils.py:1327] 2021-03-06 17:57:25,423 >> https://huggingface.co/bert-base-cased/resolve/main/config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmprn2z2ni8


...

[INFO|run_glue.py:365] 2021-03-06 18:01:57,282 >> Sample 1309 of the training set: {'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'idx': 3567, 'input_ids': [101, 153, 10783, 3663, 20595, 2349, 18784, 1667, 20575, 13225, 117, 1150, 11526, 1103, 3785, 8011, 1114, 1109, 5334, 2381, 1135, 1732, 1105, 3100, 25911, 156, 5674, 2723, 2977, 4242, 136, 102, 1622, 5259, 25795, 1176, 107, 1109, 5334, 2381, 1135, 1732, 107, 113, 3130, 114, 117, 107, 3100, 25911, 156, 5674, 2723, 2977, 4242, 136, 107, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

[INFO|trainer.py:484] 2021-03-06 18:01:57,644 >> The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: idx, sentence1, sentence2.
[INFO|trainer.py:484] 2021-03-06 18:01:57,645 >> The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: idx, sentence1, sentence2.
[INFO|trainer.py:934] 2021-03-06 18:01:57,905 >> ***** Running training *****
[INFO|trainer.py:935] 2021-03-06 18:01:57,906 >>   Num examples = 1990
[INFO|trainer.py:936] 2021-03-06 18:01:57,907 >>   Num Epochs = 3
[INFO|trainer.py:937] 2021-03-06 18:01:57,915 >>   Instantaneous batch size per device = 32
[INFO|trainer.py:938] 2021-03-06 18:01:57,917 >>   Total train batch size (w. parallel, distributed & accumulation) = 32
[INFO|trainer.py:939] 2021-03-06 18:01:57,918 >>   Gradient Accumulation steps = 1
[INFO|trainer.py:940] 2021-03-06 18:01:57,920

Step,Training Loss


[INFO|trainer.py:1117] 2021-03-06 18:04:06,588 >> 

Training completed. Do not forget to share your model on huggingface.co/models =)


[INFO|trainer.py:1535] 2021-03-06 18:04:06,815 >> Saving model checkpoint to checkpoint
[INFO|configuration_utils.py:312] 2021-03-06 18:04:06,819 >> Configuration saved in checkpoint/config.json
[INFO|modeling_utils.py:825] 2021-03-06 18:04:08,015 >> Model weights saved in checkpoint/pytorch_model.bin
[INFO|tokenization_utils_base.py:1910] 2021-03-06 18:04:08,020 >> tokenizer config file saved in checkpoint/tokenizer_config.json
[INFO|tokenization_utils_base.py:1916] 2021-03-06 18:04:08,022 >> Special tokens file saved in checkpoint/special_tokens_map.json
[INFO|run_glue.py:423] 2021-03-06 18:04:08,063 >> ***** Train results *****
[INFO|run_glue.py:425] 2021-03-06 18:04:08,064 >>   epoch = 3.0
[INFO|run_glue.py:425] 2021-03-06 18:04:08,065 >>   init_mem_cpu_alloc_delta = 53685
[INFO|run_glue.py:425] 2021-03-06 18:04:08,066 >>   init_mem_cpu_peaked_delt

[INFO|run_glue.py:449] 2021-03-06 18:04:19,919 >> ***** Eval results None *****
[INFO|run_glue.py:451] 2021-03-06 18:04:19,920 >>   epoch = 3.0
[INFO|run_glue.py:451] 2021-03-06 18:04:19,921 >>   eval_accuracy = 0.7652711272239685
[INFO|run_glue.py:451] 2021-03-06 18:04:19,928 >>   eval_loss = 0.5069981217384338
[INFO|run_glue.py:451] 2021-03-06 18:04:19,934 >>   eval_mem_cpu_alloc_delta = 81219
[INFO|run_glue.py:451] 2021-03-06 18:04:19,936 >>   eval_mem_cpu_peaked_delta = 116544
[INFO|run_glue.py:451] 2021-03-06 18:04:19,937 >>   eval_mem_gpu_alloc_delta = 0
[INFO|run_glue.py:451] 2021-03-06 18:04:19,938 >>   eval_mem_gpu_peaked_delta = 34663936
[INFO|run_glue.py:451] 2021-03-06 18:04:19,939 >>   eval_runtime = 11.5402
[INFO|run_glue.py:451] 2021-03-06 18:04:19,942 >>   eval_samples_per_second = 126.254


We're basically done now that we've gotten the losses on each block. If the model training function (`train_model.main()`) returned a mean squared error loss values instead of NLL (e.g., for regression), we'll need to convert the loss values to NLLs:

In [14]:
def mse2nll(nll, std_dev=1.):
    """
    Utility to convert expected mean squared error (MSE) to expected NLL.
    Here, MSE = (y' - y)^2, where y' is the predicted label and y is the true label.
    We treat all regression/MSE predictions as a mean with a fixed std_dev.
    We use std_dev=1 as a default, but other values may work better, e.g., if chosen on dev.
    """
    return (nll / (2. * (std_dev ** 2))) + math.log(std_dev * math.sqrt(2 * math.pi))

if mse:
    nlls = [mse2nll(mse) for mse in nlls]

Finally, we'll compute the per-sample codelengths (in bits) for different blocks (in order from earliest to latest):

In [15]:
codelengths = [nll / math.log(2) for nll in nlls]
print('Codelengths:', codelengths)

Codelengths: [1.0, 0.8554979437184423, 0.7470464580062841, 0.93560875236022, 0.8891608818937488, 0.8422681783008208, 0.8201923132534198, 0.7674828124430099, 0.7314436759720575]


To get MDL, let's compute the number of examples that were in each block:

In [16]:
block_sizes = [(block_start - block_end) for block_start, block_end in zip(block_start_idxs[1:], block_start_idxs[:-1])]
print('Block Sizes:', block_sizes)

Block Sizes: [64, 42, 70, 116, 193, 319, 529, 878, 1457]


Then, we can compute the codelength for sending each block and sum those values to get MDL:

In [17]:
mdl = sum(block_size * per_sample_codelength for block_size, per_sample_codelength in zip(block_sizes, codelengths))
print('MDL:', round(mdl), 'bits')

MDL: 2874 bits



And that's how you compute MDL! To conduct RDA, just modify the dataset inputs (where described above) and see how the MDL changes. Now you're ready to analyze your own datasets 🥳